In [3]:
import pandas as pd
import numpy as np

# Load enhanced dataset
df = pd.read_csv("../datasets/enhanced_growth_rates_emissions_energy_prod_income_level_country_df.csv")

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")

# Focus on key GMM-related indicators
key_series = [
    'Agriculture, forestry, and fishing, value added (annual % growth)',
    'Industry (including construction), value added (annual % growth)',
    'Services, value added (annual % growth)',
    'Renewable energy consumption (% of total final energy consumption)',
    'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
    'Access to electricity (% of population)',
    'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)',
    'GDP growth (annual %)'
]

df_gmm_base = df[df["Series Name"].isin(key_series)].copy()

# Pivot dataset: rows = (country, year), columns = indicator values
df_panel = df_gmm_base.melt(
    id_vars=["Country Name", "Country Code", "Income Level", "Series Name"],
    var_name="year",
    value_name="value"
)

# Fix: Handle NaN values before converting to int
print("Converting year column...")
print(f"Unique year values before processing: {df_panel['year'].unique()[:10]}")

# Extract year and handle NaN values
df_panel["year_extracted"] = df_panel["year"].astype(str).str.extract(r'(\d{4})')

# Drop rows where year extraction failed (NaN)
df_panel = df_panel.dropna(subset=['year_extracted'])

# Now convert to int (no NaN values)
df_panel["year"] = df_panel["year_extracted"].astype(int)

# Drop the temporary column
df_panel = df_panel.drop('year_extracted', axis=1)

print(f"Year range: {df_panel['year'].min()} - {df_panel['year'].max()}")

# Pivot to wide format: 1 row per (Country, year), columns = indicators
df_panel = df_panel.pivot_table(
    index=["Country Name", "Country Code", "Income Level", "year"],
    columns="Series Name",
    values="value"
).reset_index()

# Rename key columns for simplicity
rename_map = {
    'Agriculture, forestry, and fishing, value added (annual % growth)': 'AgriGrowth',
    'Industry (including construction), value added (annual % growth)': 'IndGrowth',
    'Services, value added (annual % growth)': 'ServGrowth',
    'Renewable energy consumption (% of total final energy consumption)': 'REC',
    'Energy intensity level of primary energy (MJ/$2017 PPP GDP)': 'EI',
    'Access to electricity (% of population)': 'AccessElec',
    'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)': 'PM2.5',
    'GDP growth (annual %)': 'GDPgrowth'
}

df_panel.rename(columns=rename_map, inplace=True)

# Add data quality summary
print(f"\nFinal panel dataset:")
print(f"Shape: {df_panel.shape}")
print(f"Countries: {df_panel['Country Name'].nunique()}")
print(f"Years: {df_panel['year'].min()} - {df_panel['year'].max()}")

# Save cleaned GMM base panel
df_panel.to_csv("../datasets/growth_rates_energy_gmm_base.csv", index=False)
print("✅ Saved cleaned GMM base panel to: growth_rates_energy_gmm_base.csv")

Dataset loaded successfully!
Shape: (10355, 40)
Converting year column...
Unique year values before processing: ['Series Code' '1989_1989' '1990_1990' '1991_1991' '1992_1992' '1993_1993'
 '1994_1994' '1995_1995' '1996_1996' '1997_1997']
Year range: 1989 - 2023

Final panel dataset:
Shape: (4618, 12)
Countries: 133
Years: 1989 - 2023
✅ Saved cleaned GMM base panel to: growth_rates_energy_gmm_base.csv
